In [3]:
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, ExpSineSquared, DotProduct, ConstantKernel
from autoemulate.utils import select_kernel

In [4]:
from autoemulate.simulations.projectile import simulate_projectile_multioutput
from autoemulate.experimental_design import LatinHypercube
from autoemulate.simulations.epidemic import simulate_epidemic
from autoemulate.compare import AutoEmulate
import numpy as np

lhd = LatinHypercube([(-5., 1.), (0., 1000.)]) # (upper, lower) bounds for each parameter
X = lhd.sample(100)
y = np.array([simulate_projectile_multioutput(x) for x in X])
em = AutoEmulate()
em.setup(X, y)
### Example
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
mmscaler = MinMaxScaler()
pca = PCA(0.99)
kfold = KFold(n_splits=3, shuffle=False)
em = AutoEmulate()
em.setup(X, y, scale=True, scaler=mmscaler, cross_validator=kfold)
best_model = em.compare()
pred_mean, pred_std = best_model.predict(X, return_std=True)
pred_var = np.square(pred_std)  # Convert std to variance
expectations = (pred_mean, pred_var)  

/Users/mfamili/work/autoemulate/autoemulate/compare.py:8: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


,Values
Simulation input shape (X),"(100, 2)"
Simulation output shape (y),"(100, 2)"
Proportion of data for testing (test_set_size),0.2
Scale input data (scale),True
Scaler (scaler),StandardScaler
Do hyperparameter search (param_search),False
Reduce dimensionality (reduce_dim),False
Cross validator (cross_validator),KFold
Parallel jobs (n_jobs),1


,Values
Simulation input shape (X),"(100, 2)"
Simulation output shape (y),"(100, 2)"
Proportion of data for testing (test_set_size),0.2
Scale input data (scale),True
Scaler (scaler),MinMaxScaler
Do hyperparameter search (param_search),False
Reduce dimensionality (reduce_dim),False
Cross validator (cross_validator),KFold
Parallel jobs (n_jobs),1


Initializing:   0%|          | 0/8 [00:00<?, ?it/s]

In [5]:
kernel = select_kernel('RBF')
type(kernel)

sklearn.gaussian_process.kernels.RBF

In [6]:
from sklearn.gaussian_process.kernels import RBF
import numpy as np



In [9]:
import torch

kernel = select_kernel('RBF', length_scale=5000.0)

obs = [(1500, 53, 2, 3), (200, 80, 4 , 5)]

obs_mean, obs_var = np.array(obs)
if obs_mean is not list:
    obs_mean = [obs_mean]
if obs_var is not list:
    obs_var = np.array([obs_var])
    
obs_mean = torch.tensor(obs_mean, dtype=torch.float32, requires_grad=True)
obs_var = torch.tensor(obs_var, dtype=torch.float32, requires_grad=True)
extended_pred_mean = np.tile(pred_mean, 2)
extended_pred_mean
extended_pred_mean = np.tile(pred_mean, 2)
extended_pred_mean.shape
K = kernel(extended_pred_mean[0].reshape(-1, 1))  # X is the input data
K

array([[1.        , 0.99991439, 1.        , 0.99991439],
       [0.99991439, 1.        , 0.99991439, 1.        ],
       [1.        , 0.99991439, 1.        , 0.99991439],
       [0.99991439, 1.        , 0.99991439, 1.        ]])

In [8]:
Sigma = K  # Combine cross-output and input covariance
noise_term = 1e-5 * torch.eye(Sigma.shape[0])  # Add numerical stability
Sigma = noise_term + Sigma

# Compute the inverse and log determinant of Σ(θ)
Sigma_inv = torch.inverse(Sigma)
log_det_Sigma = torch.logdet(Sigma)

# Compute the difference between observed and predicted means
diff = (obs_mean - torch.tensor(extended_pred_mean[0], requires_grad=True)).reshape(-1, 1)
print(diff.shape)
# Compute the quadratic term: (y - m(θ))^T Σ(θ)^(-1) (y - m(θ))
quad_term = torch.matmul(diff.T, torch.matmul(Sigma_inv, diff))

# Negative log-likelihood
nll = 0.5 * (quad_term + log_det_Sigma + len(diff) * torch.log(torch.tensor(2 * torch.pi)))



torch.Size([4, 1])


In [11]:
gp = em.get_model("gp")
kernel = gp.named_steps["model"].model_.module_.covar_module.base_kernel


kernel



AdditiveKernel(
  (kernels): ModuleList(
    (0): RBFKernel(
      (raw_lengthscale_constraint): Positive()
    )
    (1): ConstantKernel(
      (raw_constant_constraint): Positive()
    )
  )
)